# Prepare the data

In [1]:
from pyspark.sql import SparkSession


spark = (
    SparkSession.builder
    .config("spark.executor.memory", "16g")
    .config("spark.driver.memory", "16g")
    .config("spark.jars", "postgresql-42.7.1.jar")
    .getOrCreate()
)

23/12/28 15:05:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = spark.read.format("jdbc").options(
    url="jdbc:postgresql://localhost:5432/jrdb",
    dbtable="jrdb_curated.features",
    user="admin",
    password="admin",
    driver="org.postgresql.Driver",
).load()

In [3]:
feature_dtypes = {
    # "レースキー",
    # "馬番",
    "枠番": "category",
    "場名": "category",
    # "年月日",
    # "頭数",
    "四半期": "category",
    # "単勝的中",
    # "単勝払戻金",
    # "複勝的中",
    # "複勝払戻金",
    # "本賞金",
    # "収得賞金",
    # "血統登録番号",
    "瞬発戦好走馬_芝": "category",
    "消耗戦好走馬_芝": "category",
    "瞬発戦好走馬_ダート": "category",
    "消耗戦好走馬_ダート": "category",
    "瞬発戦好走馬_総合": "category",
    "消耗戦好走馬_総合": "category",
    "性別": "category",
    "トラック種別": "category",
    "前日_芝馬場差": "float",
    "前日_ダ馬場差": "float",
    "前日_ＩＤＭ": "float",
    "前日_脚質": "category",
    "前日_単勝オッズ": "float",
    "前日_複勝オッズ": "float",
    "直前_ＩＤＭ": "float",
    "直前_騎手指数": "float",
    "直前_情報指数": "float",
    "直前_オッズ指数": "float",
    "直前_パドック指数": "float",
    "直前_脚元情報": "category",
    "直前_天候": "category",
    "直前_単勝オッズ": "float",
    "直前_複勝オッズ": "float",
    "前走トップ3": "category",
    "前走枠番": "category",
    "入厩何日前": "float",
    "入厩15日未満": "category",
    "入厩35日以上": "category",
    "馬体重": "float",
    "馬体重増減": "float",
    "距離": "float",
    "前走距離差": "float",
    # "馬具コード",
    # "年齢",
    "4歳以下": "category",
    "4歳以下頭数": "float",
    "4歳以下割合": "float",
    "レース数": "float",
    "1位完走": "float",
    "トップ3完走": "float",
    "1位完走率": "float",
    "トップ3完走率": "float",
    "場所レース数": "float",
    "場所1位完走": "float",
    "場所トップ3完走": "float",
    "場所1位完走率": "float",
    "場所トップ3完走率": "float",
    "トラック種別レース数": "float",
    "トラック種別1位完走": "float",
    "トラック種別トップ3完走": "float",
    "トラック種別1位完走率": "float",
    "トラック種別トップ3完走率": "float",
    "馬場状態レース数": "float",
    "馬場状態1位完走": "float",
    "馬場状態トップ3完走": "float",
    "馬場状態1位完走率": "float",
    "馬場状態トップ3完走率": "float",
    "距離レース数": "float",
    "距離1位完走": "float",
    "距離トップ3完走": "float",
    "距離1位完走率": "float",
    "距離トップ3完走率": "float",
    "四半期レース数": "float",
    "四半期1位完走": "float",
    "四半期トップ3完走": "float",
    "四半期1位完走率": "float",
    "四半期トップ3完走率": "float",
    "騎手レース数": "float",
    "騎手1位完走": "float",
    "騎手トップ3完走": "float",
    "騎手1位完走率": "float",
    "騎手トップ3完走率": "float",
    "騎手場所レース数": "float",
    "騎手場所1位完走": "float",
    "騎手場所トップ3完走": "float",
    "騎手場所1位完走率": "float",
    "騎手場所トップ3完走率": "float",
    "騎手距離レース数": "float",
    "騎手距離1位完走": "float",
    "騎手距離トップ3完走": "float",
    "騎手距離1位完走率": "float",
    "騎手距離トップ3完走率": "float",
    "調教師レース数": "float",
    "調教師1位完走": "float",
    "調教師トップ3完走": "float",
    "調教師1位完走率": "float",
    "調教師トップ3完走率": "float",
    "調教師場所レース数": "float",
    "調教師場所1位完走": "float",
    "調教師場所トップ3完走": "float",
    "調教師場所1位完走率": "float",
    "調教師場所トップ3完走率": "float",
    "過去3走順位平方和": "float",
    "過去5走勝率": "float",
    "過去5走トップ3完走率": "float",
    "騎手過去5走勝率": "float",
    "騎手過去5走トップ3完走率": "float",
}

numeric_features = [
    field for field, dtype in feature_dtypes.items() if dtype in ["float"]
]

categorical_features_ordinal = [
]

categorical_features_onehot = [
    field for field, dtype in feature_dtypes.items() if dtype in ["category"]
]

label = "複勝的中"

In [4]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Load a sample dataset
data = df.toPandas().astype(feature_dtypes)
X = data[numeric_features + categorical_features]
y = data[label]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

23/12/28 15:05:36 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


# Define objective function

In [5]:
from hyperopt import hp, STATUS_OK
from sklearn.metrics import accuracy_score

def objective(params):
    numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])
    categorical_transformer = Pipeline(steps=[("onehot", OneHotEncoder())])
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
        ]
    )
    clf = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", lgb.LGBMClassifier(**params))])
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    # We aim to maximize accuracy, hence negative sign
    return {'loss': -accuracy, 'status': STATUS_OK}


# Define hyperparameter space

In [6]:
from hyperopt import hp

# Define the hyperparameter space
space = {
    'boosting_type': hp.choice('boosting_type', ['gbdt', 'dart', 'goss']),  # Gradient Boosting Decision Tree, Dropouts meet Multiple Additive Regression Trees, Gradient-based One-Side Sampling
    'num_leaves': hp.choice('num_leaves', range(20, 151)),  # Number of leaves in one tree
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),  # Learning rate or shrinkage rate
    'n_estimators': hp.choice('n_estimators', range(100, 1001)),  # Number of boosted trees to fit
    'max_depth': hp.choice('max_depth', range(5, 31)),  # Maximum tree depth
    'min_child_weight': hp.uniform('min_child_weight', 0.1, 10),  # Minimum sum of instance weight (hessian) needed in a child (leaf)
    'subsample': hp.uniform('subsample', 0.5, 1),  # Subsample ratio of the training instance
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),  # Subsample ratio of columns when constructing each tree
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),  # L1 regularization term on weights
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),  # L2 regularization term on weights
    "verbose": -1,
    "seed": 80
}


# space = {
#     "metric": "binary_error",
#     # "metric": "multi_error",
#     "boosting": "gbdt",
#     "min_data_in_leaf": 99,
#     "n_estimators": 300,
#     "objective": "binary",
#     # "objective": "multiclass",
#     # "num_class": 14,
#     "seed": 80,
#     "learning_rate": 0.03,  # hp.choice('learning_rate', [0.05, .1, .3]),
#     "max_depth": hp.choice("max_depth", range(6, 25)),
#     "num_leaves": hp.choice("num_leaves", range(20, 100, 10)),
#     "subsample": hp.uniform("subsample", 0.5, 1),
#     "feature_fraction": hp.uniform("feature_fraction", 0.5, 1),  # colsample_bytree
#     "reg_alpha": hp.uniform("reg_alpha", 0, 1),
#     "reg_lambda": hp.uniform("reg_lambda", 0, 1),
#     "min_child_samples": hp.choice(
#         "min_child_samples", range(10, 100, 10)
#     ),
#     "verbose": -1,
# }

# Run optimization

In [11]:
spark.stop()

In [12]:
from hyperopt import fmin, tpe, Trials, SparkTrials

spark = (
    SparkSession.builder
    .config("spark.executor.memory", "5g")
    .config("spark.driver.memory", "5g")
    .config("spark.jars", "postgresql-42.7.1.jar")
    .getOrCreate()
)

# trials = Trials()
# Example for using SparkTrials
trials = SparkTrials(parallelism=5, spark_session=spark)
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials)

print("Best hyperparameters:", best)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

  1%|          | 1/100 [01:35<2:37:40, 95.56s/trial, best loss: -0.9599567232624758]

  2%|▏         | 2/100 [01:55<1:23:36, 51.19s/trial, best loss: -0.9611943723848515]

  3%|▎         | 3/100 [02:24<1:06:28, 41.12s/trial, best loss: -0.9611943723848515]

  4%|▍         | 4/100 [03:14<1:10:55, 44.33s/trial, best loss: -0.9611943723848515]

...

 98%|█████████▊| 98/100 [3:25:47<04:17, 128.67s/trial, best loss: -0.9618009024183336]

 99%|█████████▉| 99/100 [3:26:04<01:35, 95.21s/trial, best loss: -0.9618009024183336] 

100%|██████████| 100/100 [3:29:10<00:00, 125.50s/trial, best loss: -0.9618009024183336]

Total Trials: 100: 100 succeeded, 0 failed, 0 cancelled.


Best hyperparameters: {'boosting_type': 1, 'colsample_bytree': 0.8369799191033762, 'learning_rate': 0.14408193699659572, 'max_depth': 13, 'min_child_weight': 1.3239044242562388, 'n_estimators': 885, 'num_leaves': 95, 'reg_alpha': 0.693583254475872, 'reg_lambda': 0.8707167432885, 'subsample': 0.7529152300659998}


In [13]:
print(best)

{'boosting_type': 1, 'colsample_bytree': 0.8369799191033762, 'learning_rate': 0.14408193699659572, 'max_depth': 13, 'min_child_weight': 1.3239044242562388, 'n_estimators': 885, 'num_leaves': 95, 'reg_alpha': 0.693583254475872, 'reg_lambda': 0.8707167432885, 'subsample': 0.7529152300659998}


# Train final model

In [30]:
best_params = space.copy()
best_params.update(best)

best_params = {
    "boosting_type": "gbdt",  # 1
    "colsample_bytree": 0.8369799191033762,
    "learning_rate": 0.14408193699659572,
    "max_depth": 13,
    "min_child_weight": 1.3239044242562388,
    "n_estimators": 885,
    "num_leaves": 95,
    "reg_alpha": 0.693583254475872,
    "reg_lambda": 0.8707167432885,
    "subsample": 0.7529152300659998,
}

final_model = lgb.LGBMClassifier(**best_params)
final_model.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 206878, number of negative: 769165
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.069151 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11592
[LightGBM] [Info] Number of data points in the train set: 976043, number of used features: 98
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211956 -> initscore=-1.313176
[LightGBM] [Info] Start training from score -1.313176
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

LGBMClassifier(colsample_bytree=0.8369799191033762,
               learning_rate=0.14408193699659572, max_depth=13,
               min_child_weight=1.3239044242562388, n_estimators=885,
               num_leaves=95, reg_alpha=0.693583254475872,
               reg_lambda=0.8707167432885, subsample=0.7529152300659998)

# Evaluate model

In [33]:
final_pred = final_model.predict(X_test)
final_accuracy = accuracy_score(y_test, final_pred)
print("Final model accuracy:", final_accuracy)

Final model accuracy: 0.961259943199282


In [34]:
list(data.columns)

Index(['レースキー', '馬番', '枠番', '場名', '年月日', '頭数', '四半期', '単勝的中', '単勝払戻金', '複勝的中',
       ...
       '調教師場所レース数', '調教師場所1位完走', '調教師場所トップ3完走', '調教師場所1位完走率', '調教師場所トップ3完走率',
       '過去3走順位平方和', '過去5走勝率', '過去5走トップ3完走率', '騎手過去5走勝率', '騎手過去5走トップ3完走率'],
      dtype='object', length=111)

In [38]:
(
    X_test
    .assign(final_pred=final_pred)
    # .join(data[["レースキー", "馬番", "複勝払戻金"]], on=["レースキー", "馬番"], how="left")
    # .assign(的中=lambda df: df["final_pred"] == df["複勝的中"])
)

,前日_芝馬場差,前日_ダ馬場差,前日_ＩＤＭ,前日_単勝オッズ,前日_複勝オッズ,直前_ＩＤＭ,直前_騎手指数,直前_情報指数,直前_オッズ指数,直前_パドック指数,...,トラック種別,前日_脚質,直前_脚元情報,直前_天候,前走トップ3,前走枠番,入厩15日未満,入厩35日以上,4歳以下,final_pred
118382,-8.0,-8.0,34.0,22.4,4.3,34.0,0.4,3.7,0.0,0.0,...,ダート,先行,0,晴,False,2,False,True,True,False
724411,NaN,NaN,54.0,NaN,NaN,53.0,0.8,1.2,0.0,0.0,...,芝,先行,0,雨,True,5,NaN,NaN,True,False
636861,-6.0,-8.0,43.0,5.8,1.5,43.0,1.6,2.4,2.0,2.0,...,芝,先行,0,小雨,False,8,False,False,True,False
931865,NaN,NaN,34.0,38.7,6.8,34.0,0.3,-1.0,0.0,0.0,...,芝,好位差し,0,晴,False,5,NaN,NaN,True,True
473644,-20.0,-8.0,52.0,5.5,1.9,52.0,1.5,0.8,3.0,0.0,...,ダート,差し,2,晴,False,6,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541001,-10.0,-15.0,10.0,105.7,21.0,10.0,0.0,0.0,0.0,0.0,...,芝,追込,0,雨,False,8,False,False,True,False
1157787,-6.0,-12.0,24.0,40.7,7.3,24.0,0.2,0.0,0.0,2.4,...,芝,追込,0,曇,False,5,NaN,NaN,True,True
513018,-16.0,-12.0,44.0,27.7,4.8,44.0,0.4,1.2,2.5,0.0,...,芝,差し,0,晴,False,8,False,True,True,False
1148413,-8.0,-10.0,23.7,13.4,3.4,23.7,0.7,0.8,0.0,0.0,...,ダート,逃げ,0,晴,False,3,False,True,True,False
